## Fazendo requisições para gerar esquemas JSON
* 'http://localhost:4200/api/batch/rawschema/steps/all'

* Execultar 10 vezes

* Salvar o resultado do tempo de processamento em csv

* **Conjuntos de coleções usadas nos experimentos da abordagem em Lote**

    * **Conjunto de dados Twitter**
        * "twitter_450"
        * "twitter_1350"
        * "twitter_3150"
        * "twitter_6750"
        * "twitter_13950"
        * "twitter_28350"
        * "twitter_57150"
        * "twitter_114750"
        * "twitter_229950"
        * "twitter_460350"
        * "twitter_921150"
        * "twitter_1842750"


    * **Conjunto de dados VK**

        * "vk_450"
        * "vk_1350"
        * "vk_3150"
        * "vk_6750"
        * "vk_13950"
        * "vk_28350"
        * "vk_57150"
        * "vk_114750"
        * "vk_229950"
        * "vk_460350"
        * "vk_921150"
        * "vk_1842750"

    * **Conjunto de dados de metadados de livros**

        * "books_10000"
        * "books_30000"
        * "books_70000"
        * "books_150000"
        * "books_310000"
        * "books_630000"
        * "books_1270000"
        * "books_2550000"
        * "books_5110000"
        * "books_10230000"
        * "books_20470000"
        * "books_40950000"



#### Imports

In [ ]:
import requests
import time
from json.decoder import JSONDecodeError
import csv
import json

### Criar uma conta

In [ ]:
import requests

# Dados do usuário
user_data = {
    'username': config['username'],
    'email': config['email'],
    'password': config['password']
}

# URL do endpoint
url = 'http://localhost:4200/api/register'

# Fazendo a requisição POST
response = requests.post(url, json=user_data)

# Verificando a resposta
if response.status_code == 200:
    print('Requisição bem-sucedida!')
    print('Resposta:', response.json())  
elif response.status_code == 400:
    try:
        error_details = response.json()
        # Verifica se o erro é de duplicação de email
        if error_details.get('error', {}).get('code') == 11000 and 'email' in error_details.get('error', {}).get('keyPattern', {}):
            print('O email já está registrado.')
        else:
            print('Erro na requisição:', response.status_code)
            print('Detalhes do erro:', response.text)
    except ValueError:
        # Caso a resposta não seja um JSON válido
        print('Erro na requisição:', response.status_code)
        print('Detalhes do erro:', response.text)
else:
    print('Erro na requisição:', response.status_code)
    print('Detalhes do erro:', response.text)


### Acessar a conta criada e recuperar o token de autentificação. 

In [ ]:
# Acessar e pegar o token de autorização
import requests

# Dados de login
login_data = {
    'email': config['email'],
    'password': config['password']
}

# URL requisição
login_url = 'http://localhost:4200/api/login'  

# Fazendo a requisição de login
response = requests.post(login_url, json=login_data)

# Verificando a resposta
if response.status_code == 200:
    print('Login bem-sucedido!')
    token = response.json().get('token')
    if token:
        print('Token de acesso:', token)
    else:
        print('Token de acesso não encontrado na resposta.')
else:
    print('Erro no login:', response.json().get('message'))
    print('Código de erro:', response.json().get('code'))


### Experimentos JSD

In [ ]:
# Caminho para o arquivo de configuração
config_file_path = 'config_JSD.json'

# Carregar os dados do arquivo JSON
with open(config_file_path, 'r') as file:
    config = json.load(file)

# Númenro de repetições
num_repetitions = config['num_repetitions']

#Coleções usadas para evoluir o esquema JSON
collections_to_Batch = config['collections_to_Batch']

# Token autorização
headers_batch = {
    'Authorization': 'Bearer '+ token,
    'Content-Type': 'application/json'
}

# URL requisição
url_batch = 'http://localhost:4200/api/batch/rawschema/steps/all'


In [ ]:

# Função para realizar a requisição e medir o tempo de execução
def make_request(url, data, headers):
    start_time = time.time()
    response = requests.post(url, json=data, headers=headers)
    elapsed_time = time.time() - start_time
    
    # Imprime as informações
    print(f"URL: {url}")
    print(f"Status code: {response.status_code}")
    
    try:
        # Tenta decodificar a resposta JSON
        response_json = response.json()
        print(f"Response: {response_json}")
    except JSONDecodeError:
        # Se houver um erro ao decodificar, imprime uma mensagem indicando que não foi possível decodificar a resposta
        print("Response: Unable to decode JSON")
    
    print(f"Tempo de processamento: {elapsed_time} segundos")
    print("------------------------")
    
    return response, elapsed_time


elapsed_times_batch_ = []

with open('resultados_vk_batch___.csv', mode='w', newline='') as file:
    fieldnames = ['Iteração', 'Coleção', 'Tempo de Execução (Batch)', 'Tempo de Execução (Update)']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for iteration in range(num_repetitions):
        for collection in collections_to_Batch:
            data_batch = {
                "authentication": {
                    "authMechanism": "SCRAM-SHA-1",
                    "userName": config['database_userName'],
                    "password": config['database_password']
                },
                "port": "27017",
                "address": "localhost",
                "databaseName": database,
                "collectionName": collection,
                "userId": "65b07893d3104d1b36282d4f",
            }
            
            iteration_ = iteration + 1
            print("Iteração: ", iteration_)
            
            response_batch, elapsed_time_batch = make_request(url_batch, data_batch, headers_batch)
            
            writer.writerow({
                'Iteração': iteration + 1,
                'Coleção': collection,
                'Tempo de Execução (Batch)': elapsed_time_batch
            })
